In [1]:
import numpy as np
import matplotlib.pyplot as plt
from OpenGoddard.optimize import Problem, Guess, Condition, Dynamics

from rocket import Rocket

In [2]:
r = Rocket()

In [3]:
r

In [4]:
def og_dynamics(prob, obj, section):
    
    #extract states and controls
    s = tuple([prob.states(i, section) for i in range(5)])
    u = tuple([prob.controls(i, section) for i in range(2)])
    
    
    dx = Dynamics(prob, section)
    
    #get dynamics from the rocket
    ds = obj.vec_dynamics(s, u)
    
    for i in range(5):
        dx[i] = ds[i]
        
    return dx()

def og_equality(prob, obj):
    s = [prob.states_all_section(i) for i in range(5)]
    u = [prob.controls_all_section(i) for i in range(2)]
    
    result = Condition()
    for i in range(5):
        result.equal(s[i][0], obj.s_0[i]), #r, v, m = [r,v,m](0)
    
    for i in range(4):
        result.equal(s[i][-1], 0.) #r, v = 0
        
    return result

def og_inequality(prob, obj):
    s = [prob.states_all_section(i) for i in range(5)]
    u = [prob.controls_all_section(i) for i in range(2)]
    
    result = Condition()
    
    #require u to be bounded
    
    result.lower_bound(u[0], 0.)
    result.upper_bound(u[0], 1.)
    
    result.lower_bound(u[1], -np.pi)
    result.upper_bound(u[1], +np.pi)
    
    #bound states
    
    result.lower_bound(s[4], 0.)
    result.upper_bound(s[4], obj.m_0)
    
    
def og_cost(prob, obj):
    s = [prob.states_all_section(i) for i in range(5)]
    u = [prob.controls_all_section(i) for i in range(2)]
    tf = prob.time_final(-1)
    
    #maximize final mass, so minimize the negative
    
    m = s[-1]
    
    
    return -m[-1]/obj.m_0


    
    
    
    

In [5]:
time_init = [0.0, (2*r.z_0/r.g)]
n = [20]
num_states = [5]
num_controls = [2]
max_iteration = 20


In [6]:
prob = Problem(time_init, n, num_states, num_controls, max_iteration)
prob.dynamics = [og_dynamics]
prob.cost = og_cost
prob.equality = og_equality
prob.inequality = og_inequality

In [7]:
x_guess  = Guess.linear(prob.time_all_section, r.x_0,  0)
z_guess  = Guess.linear(prob.time_all_section, r.z_0,  0)
vx_guess = Guess.linear(prob.time_all_section, r.vx_0, 0)
vz_guess = Guess.linear(prob.time_all_section, r.vz_0, 0)
m_guess  = Guess.linear(prob.time_all_section, r.m_0,  r.m_0/2)

u1_guess = Guess.linear(prob.time_all_section, 1, 1)
u2_guess = Guess.linear(prob.time_all_section, 0, 0)

In [8]:
prob.set_states_all_section(0, x_guess)
prob.set_states_all_section(1, z_guess)
prob.set_states_all_section(2, vx_guess)
prob.set_states_all_section(3, vz_guess)
prob.set_states_all_section(4, m_guess)

prob.set_controls_all_section(0, u1_guess)
prob.set_controls_all_section(0, u2_guess)

In [9]:
prob.solve(r)

---- iteration : 1 ----
10.0


TypeError: 'numpy.float64' object is not iterable